In [ ]:
import os
import csv
import glob

root_folder = r"C:\Users\engno\Desktop\testing_data"
data_dict = {}

file_pattern = os.path.join(root_folder, '**', '*.csv')
files_to_process = [file for file in glob.glob(file_pattern, recursive=True)]

# Create a dictionary to hold values for each key_name_parts[4] value
data_dict = {}
keywords = ["PMP_DISC_PRESS","PMP_INTK_PRESS","FL_PRESS","FL_TEMP","MOT_RUN_FREQ","PHASE_B_CDH", "MOT_OUT_VOLT", "WH_PRESS","OUTPUT_AMPS"]
for file_path in files_to_process:
    filename = os.path.basename(file_path)
    key_name_parts = filename[:-4].split('_')[1:]
    
    # Extract key_name_parts[4] value
    well_name = key_name_parts[4]
    tag_name='_'.join(key_name_parts)
    print(tag_name)
    if any(keyword in tag_name for keyword in keywords):
    # if tag_name == "AM28341":
        with open(file_path, 'r') as csv_file:
            csv_reader = csv.reader(csv_file)
            try:
                header = next(csv_reader)
            except StopIteration:
                print("CSV file is empty or does not contain any data.")
                # Handle the error here, such as exiting the program or taking alternative actions.
                # You could also choose to continue without raising an error if it's appropriate for your use case.
            else:
                index_to_extract = 1
            # header = next(csv_reader)
            # index_to_extract = 1

            # Extract values, skipping rows where the specified column contains "Bad" or "I/O Timeout"
            values = [row[index_to_extract] for row in csv_reader if row[index_to_extract] not in ["Bad", "I/O Timeout"]]
                
            # Initialize the list if the key doesn't exist
            if tag_name not in data_dict:
                data_dict[tag_name] = []
                    
            # Add values to the dictionary based on key_name_parts[4] value
            data_dict[tag_name].extend(values)


In [ ]:
# data_dict.values()

In [ ]:
import time
import requests
import threading

url = "http://192.168.100.135:44372/api/pi/write/"

def send_requests(key, values):
    for value in values:
        payload = {"tag": key, "value": value}
        headers = {"Content-Type": "application/json"}
        
        response = requests.post(url, json=payload, headers=headers)
        # Print the response status and content
        print(f"POST Status for {key}: {response.status_code}")
        print(response.json())  # Assuming the response is in JSON format

def process_data(data_dict):
    threads = []

    for key, values in data_dict.items():

        thread = threading.Thread(target=send_requests, args=(key, values))
        threads.append(thread)
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()


while True:
    process_data(data_dict)
    
    # Wait for one minute before repeating the process
    time.sleep(60)